In [20]:
import hashlib
import ecdsa
import secrets
import time


nfc_private_key = secrets.randbits(256)
phone_private_key = secrets.randbits(256)

#  to hex strings
nfc_private_key_hex = hex(nfc_private_key)[2:].zfill(64)
phone_private_key_hex = hex(phone_private_key)[2:].zfill(64)

print("NFC Tag Private Key:", nfc_private_key_hex)
print("Phone Wallet Private Key:", phone_private_key_hex)

NFC Tag Private Key: 6c2b7bca294b9a21bbd0350e6b41c4e4843f5838bda41c6ae609389fac64da00
Phone Wallet Private Key: 586df52aed4f7acb2edca7015e08a73b555fbef3fb530e976a8729d80477afcd


In [21]:
def sign_message(private_key_hex, message, nonce, timestamp, chain_id, contract_address):
    private_key = ecdsa.SigningKey.from_string(bytes.fromhex(private_key_hex), curve=ecdsa.SECP256k1)
    
    # security parameters
    full_message = f"{message}|nonce:{nonce}|timestamp:{timestamp}|chain_id:{chain_id}|contract:{contract_address}"
    
    message_hash = hashlib.sha3_256(full_message.encode('utf-8')).digest()
    signature = private_key.sign(message_hash)
    return signature.hex()

# Security parameters
nonce = secrets.randbits(32)
timestamp = int(time.time())
chain_id = 1
contract_address = "0x6dcF7b72a4a0236F7FF69ed42A861B3155f4dBfB"



# Messages to sign 
nfc_message = "This is an NFC tag message"
phone_message = "This is a phone wallet message"

nfc_signature = sign_message(nfc_private_key_hex, nfc_message, nonce, timestamp, chain_id, contract_address)
phone_signature = sign_message(phone_private_key_hex, phone_message, nonce+1, timestamp, chain_id, contract_address)


print("\nNFC Tag Message:", nfc_message)
print("NFC Tag Signature:", nfc_signature)

print("\nPhone Wallet Message:", phone_message)
print("Phone Wallet Signature:", phone_signature)

# Function to verify a signature
def verify_signature(public_key_hex, message, signature_hex, nonce, timestamp, chain_id, contract_address, max_age=300):
    public_key = ecdsa.VerifyingKey.from_string(bytes.fromhex(public_key_hex), curve=ecdsa.SECP256k1)
    
    # Reconstruct full message
    full_message = f"{message}|nonce:{nonce}|timestamp:{timestamp}|chain_id:{chain_id}|contract:{contract_address}"
    
    message_hash = hashlib.sha3_256(full_message.encode('utf-8')).digest()
    
    # Check if the signature has expired
    current_time = int(time.time())
    if current_time - timestamp > max_age:
        return False, "Signature has expired"
    
    try:
        is_valid = public_key.verify(bytes.fromhex(signature_hex), message_hash)
        return is_valid, "Signature is valid" if is_valid else "Signature is invalid"
    except:
        return False, "Signature verification failed"

# Derive public keys from private keys
nfc_public_key = ecdsa.SigningKey.from_string(bytes.fromhex(nfc_private_key_hex), curve=ecdsa.SECP256k1).get_verifying_key()
phone_public_key = ecdsa.SigningKey.from_string(bytes.fromhex(phone_private_key_hex), curve=ecdsa.SECP256k1).get_verifying_key()

nfc_public_key_hex = nfc_public_key.to_string().hex()
phone_public_key_hex = phone_public_key.to_string().hex()

print("\nNFC Tag Public Key:", nfc_public_key_hex)
print("Phone Wallet Public Key:", phone_public_key_hex)

# verify signatures
nfc_verification, nfc_message = verify_signature(nfc_public_key_hex, nfc_message, nfc_signature, nonce, timestamp, chain_id, contract_address)
phone_verification, phone_message = verify_signature(phone_public_key_hex, phone_message, phone_signature, nonce+1, timestamp, chain_id, contract_address)

print("\nNFC Tag Signature Verification:", nfc_verification)
print("Phone Wallet Signature Verification:", phone_verification)


NFC Tag Message: This is an NFC tag message
NFC Tag Signature: 7986ea66a42fa1af101c997ef953574bd0936ea95225c23a591bac7f330b8296120818c6022520deda1017380662a72409f3f2eceb58cc0a405d0abff2883b77

Phone Wallet Message: This is a phone wallet message
Phone Wallet Signature: aa4f055b5bb63feaf35a588fa8be6bec8b10a8b98884a80cdf4c73a25b05962a6499cf11819568d90b864e51dc3047fd559c4e781055cac13612378140f55302

NFC Tag Public Key: 7dfe09d07126c7bfc6ebe92817d0776971cf796861ce1597ba4c951675ea10301d44ba41b6a296d73fbab46fcec4d5f687d024b93a2890f26fa1b487affb20cf
Phone Wallet Public Key: 4e5046c8f1dbd1271c898230c61aa87ff5cdd3e2caea7ad7c02d7bc03ba4e6f2b21636987c15ee4dbd33605a2c5decc4d61e97a2faceacf8904958a0b5375ce2

NFC Tag Signature Verification: True
Phone Wallet Signature Verification: True


In [ ]:
# at this point you should be able to activate the smart contract. 